# TODO

- [x] Implement action repetition every 4 frames.
- [x] Change model architecture
- [x] Huber loss in gradient step.
- [x] Save model and optimizer.
- [x] Time training.
- [x] Use tricks from paper (data collection, rmsprop)
- [x] Optimize batch loss calc
- [ ] Save statistics from paper.
- [ ] Pick out frames with obvious Q vals and graph them

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

## DQN Algo Outline

In [3]:
from collections import namedtuple
import random
import time

import torch
from torch import nn
from torch import optim
from torchvision import transforms
import numpy as np

import gym
from src import agent, train, utils

In [4]:
random.seed(0)

## Explore

### Collect 10 random frames

In [5]:
def initialize(replay_mem_size, batch_size):
    agt = agent.DQNAgent()
    replay_mem = utils.ReplayMemory(replay_mem_size, batch_size)
    obs_history = utils.ObsHistory()
    env = gym.envs.make('PongNoFrameskip-v4')
    train_stats = TrainingStats(agt)
    
    return agt, replay_mem, obs_history, env, train_stats


def mini_batch_to_tensor(mini_batch, agt):
    phi, a, r, phi_1, dones = mini_batch
    
    phi = phi.to(agt.device, non_blocking=True)
    phi_1 = phi.to(agt.device, non_blocking=True).detach()
    a = a.to(agt.device, torch.long, non_blocking=True)
    r = r.to(agt.device, torch.float, non_blocking=True)
    dones = dones.to(agt.device, torch.float, non_blocking=True)
    
    return phi, a, r, phi_1, dones
    
    
def mini_batch_loss(mini_batch, gamma, agt):
    phi, a, r, phi_1, dones = mini_batch_to_tensor(mini_batch, agt)
    
    q_phi_1 = agt.target(phi_1).max(1)[0] * (1 - dones)
    y = (r + gamma * q_phi_1).detach()
    
    q_trans = agt.qnet(phi).gather(1, a.unsqueeze(1))

    loss = nn.SmoothL1Loss()
    return loss(y, q_trans)


def gradient_step(replay_mem, agt, gamma):
    if replay_mem.size > replay_mem.sample_size + 3:
        mini_batch = replay_mem.sample()
        
        agt.optimizer.zero_grad()
        t0 = time.time()
        loss = mini_batch_loss(mini_batch, gamma, agt)
        t1 = time.time()
        print('loss: {}'.format(t1-t0))
        loss.backward()
        agt.optimizer.step()
        t2 = time.time()
        print('backprop: {}'.format(t2-t1))
        print('=====================================')
        
        return loss.item()

def save_params(agt, episodes, total_steps, save_path):
    torch.save({
        'model_state_dict': agt.qnet.state_dict(),
        'optimizer_state_dict': agt.optimizer.state_dict(),
        'episodes': episodes,
        'total_steps': total_steps
    }, save_path)

    
def load_params(agt, load_path):
    checkpoint = torch.load(checkpoint_path)
    
    agt.qnet.load_state_dict(checkpoint['model_state_dict'])
    agt.opimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    return chekpoint['episodes'], checkpoint['total_steps']


def reset_episode(env, obs_history):
    s_t = env.reset()
    obs_history.reset(s_t)
    done = False
    
    return s_t, done

In [6]:
def get_rand_transitions(k, n):
    env = gym.envs.make('PongNoFrameskip-v4')
    s_t = env.reset()
    
    replay_mem = utils.ReplayMemory(n, k)
    
    for i in range(n):
        a_t = random.randrange(env.action_space.n)
        s_t1, r_t, done, _ = env.step(a_t)
        replay_mem.store(s_t, a_t, r_t, done)
        
        s_t = s_t1
        
        if done:
            s_t = env.reset()
            
    return replay_mem.sample()

In [7]:
def frames_to_phi(frames):
    frames = [utils.process_frame(s) for s in frames]
    phi = torch.cat(frames)

    return phi

def get_rand_phis(k, n):
    frames = []
    env = gym.envs.make('PongNoFrameskip-v4')
    env.reset()

    for i in range(n):
        a = random.randrange(env.action_space.n)
        s_t1, r_t, done, _ = env.step(a)
        frames.append(s_t1)

        if done:
            s_t = env.reset()

    idxs = random.sample(range(3, n), k)
    phis = [frames_to_phi(frames[i-3:i+1]) for i in idxs]

    return phis

In [8]:
class TrainingStats:
    
    def __init__(self, agt):
        self.agt = agt
        self.total_steps = 0
        self.ep_rewards = []
        self.steps_per_ep = []
        self.benchmark_qvals = []
        self.benchmark_frames = \
        torch.stack(get_rand_phis(10, 10000)).to(agt.device)
        self.t = time.time()
        
    def store(self, ep_reward, ep_steps, episode_num, total_steps):
        self.ep_rewards.append(ep_reward)
        self.steps_per_ep.append(ep_steps)
        self.total_steps += ep_steps
        
        
        avg_qvals = self.get_frames_avg_qval()
        self.benchmark_qvals.append(avg_qvals)
        ep_dur = time.time() - self.t
        self.t = time.time()
        
        if episode_num % 50 == 49:
            save_params(self.agt, episode_num, total_steps, 
                        'dqn_agt_{}.pt'.format(episode_num+1))
            
        
        print('Episode {}:'.format(episode_num))
        print('Reward: {}'.format(ep_reward))
        print('Total steps: {}'.format(self.total_steps))
        print('Avg qvals: {:.5f}'.format(avg_qvals))
        print('Duration: {:.2f}'.format(ep_dur))
        print('===========================================')
        

    def get_frames_avg_qval(self):
        qvals = self.agt.target(self.benchmark_frames).max(1)[0]

        return torch.mean(qvals).item()

In [9]:
gamma = .99
replay_mem_size = int(4e5)
batch_size = 32
num_episodes = int(1e3)

ep_rewards = []
ep_avg_train_losses = []
steps_per_ep = []
benchmark_qvals = []

In [10]:
torch.cuda.is_available()

True

In [11]:
agt, replay_mem, obs_history, env, train_stats = \
    initialize(replay_mem_size, batch_size)

total_steps = 0

for episode in range(num_episodes):
    s_t, done = reset_episode(env, obs_history)
    a_t = 0
    
    ep_reward = 0
    ep_steps = 0
    
    while not done:
        if ep_steps % 4 == 0: # select action every 4 frames
            phi_t = obs_history.get_phi()
            a_t = agt.act(phi_t)
        s_t1, r_t, done, _ = env.step(a_t)
        
        obs_history.store(s_t1)
        replay_mem.store(s_t, a_t, r_t, done)
        s_t = s_t1
        
        if total_steps > 50000:
            loss_val = gradient_step(replay_mem, agt, gamma)
        
        if total_steps % 10000 == 0:
            agt.target.load_state_dict(agt.qnet.state_dict())
            

        ep_reward += r_t
        ep_steps += 1
        total_steps += 1
        
    
    train_stats.store(ep_reward, ep_steps, episode, 
                      total_steps)

Episode 0:
Reward: -21.0
Total steps: 3168
Avg qvals: 0.05025
Duration: 2.82
Episode 1:
Reward: -20.0
Total steps: 7910
Avg qvals: 0.05025
Duration: 3.90
Episode 2:
Reward: -20.0
Total steps: 11708
Avg qvals: 0.05025
Duration: 3.12
Episode 3:
Reward: -18.0
Total steps: 15988
Avg qvals: 0.05025
Duration: 3.57
Episode 4:
Reward: -21.0
Total steps: 19044
Avg qvals: 0.05025
Duration: 2.55
Episode 5:
Reward: -21.0
Total steps: 22692
Avg qvals: 0.05025
Duration: 2.94
Episode 6:
Reward: -21.0
Total steps: 26108
Avg qvals: 0.05025
Duration: 3.00
Episode 7:
Reward: -21.0
Total steps: 29404
Avg qvals: 0.05025
Duration: 3.06
Episode 8:
Reward: -19.0
Total steps: 33626
Avg qvals: 0.05025
Duration: 3.43
Episode 9:
Reward: -21.0
Total steps: 37258
Avg qvals: 0.05025
Duration: 3.00
Episode 10:
Reward: -20.0
Total steps: 41357
Avg qvals: 0.05025
Duration: 3.80
Episode 11:
Reward: -20.0
Total steps: 44724
Avg qvals: 0.05025
Duration: 3.15
Episode 12:
Reward: -21.0
Total steps: 48937
Avg qvals: 0.05025


loss: 0.002452850341796875
backprop: 0.002238750457763672
loss: 0.001905679702758789
backprop: 0.0017590522766113281
loss: 0.0020751953125
backprop: 0.001999378204345703
loss: 0.0017733573913574219
backprop: 0.0017042160034179688
loss: 0.0016465187072753906
backprop: 0.001674652099609375
loss: 0.001775503158569336
backprop: 0.002304553985595703
loss: 0.0018355846405029297
backprop: 0.0018298625946044922
loss: 0.001722574234008789
backprop: 0.0018410682678222656
loss: 0.0016589164733886719
backprop: 0.0016484260559082031
loss: 0.0017790794372558594
backprop: 0.0017695426940917969
loss: 0.0017538070678710938
backprop: 0.0018649101257324219
loss: 0.0016541481018066406
backprop: 0.00180816650390625
loss: 0.0016672611236572266
backprop: 0.0019266605377197266
loss: 0.0018074512481689453
backprop: 0.0018117427825927734
loss: 0.0017542839050292969
backprop: 0.001756429672241211
loss: 0.001821756362915039
backprop: 0.0017282962799072266
loss: 0.0017566680908203125
backprop: 0.001932144165039062

backprop: 0.002384185791015625
loss: 0.0016112327575683594
backprop: 0.0017805099487304688
loss: 0.0015897750854492188
backprop: 0.0017476081848144531
loss: 0.0018417835235595703
backprop: 0.0016927719116210938
loss: 0.0015530586242675781
backprop: 0.0017201900482177734
loss: 0.001575469970703125
backprop: 0.001737356185913086
loss: 0.001538991928100586
backprop: 0.0019795894622802734
loss: 0.0015444755554199219
backprop: 0.0017871856689453125
loss: 0.001538991928100586
backprop: 0.0017559528350830078
loss: 0.0015411376953125
backprop: 0.0017402172088623047
loss: 0.001543283462524414
backprop: 0.0018353462219238281
loss: 0.0015587806701660156
backprop: 0.0017104148864746094
loss: 0.0015954971313476562
backprop: 0.0017244815826416016
loss: 0.0016193389892578125
backprop: 0.0017325878143310547
loss: 0.0015423297882080078
backprop: 0.0017213821411132812
loss: 0.0015840530395507812
backprop: 0.0017066001892089844
loss: 0.0015513896942138672
backprop: 0.0017833709716796875
loss: 0.001543521

loss: 0.0018262863159179688
backprop: 0.002088308334350586
loss: 0.002031564712524414
backprop: 0.002028226852416992
loss: 0.0021920204162597656
backprop: 0.0015900135040283203
loss: 0.001634359359741211
backprop: 0.0016558170318603516
loss: 0.001920938491821289
backprop: 0.003025531768798828
loss: 0.002141714096069336
backprop: 0.002180337905883789
loss: 0.002499103546142578
backprop: 0.0020902156829833984
loss: 0.002038717269897461
backprop: 0.0017867088317871094
loss: 0.0016558170318603516
backprop: 0.0017578601837158203
loss: 0.0018715858459472656
backprop: 0.0026960372924804688
loss: 0.002260923385620117
backprop: 0.0017004013061523438
loss: 0.002084493637084961
backprop: 0.0023207664489746094
loss: 0.002855539321899414
backprop: 0.0021049976348876953
loss: 0.0016350746154785156
backprop: 0.0016360282897949219
loss: 0.0019426345825195312
backprop: 0.0029251575469970703
loss: 0.002745389938354492
backprop: 0.0024271011352539062
loss: 0.0020723342895507812
backprop: 0.00249242782592

loss: 0.002166748046875
backprop: 0.0019681453704833984
loss: 0.001600503921508789
backprop: 0.0017380714416503906
loss: 0.0016067028045654297
backprop: 0.0018055438995361328
loss: 0.001661062240600586
backprop: 0.0016965866088867188
loss: 0.0016431808471679688
backprop: 0.0017120838165283203
loss: 0.0016102790832519531
backprop: 0.0017092227935791016
loss: 0.0015707015991210938
backprop: 0.0017092227935791016
loss: 0.0015382766723632812
backprop: 0.0017104148864746094
loss: 0.0015366077423095703
backprop: 0.001714944839477539
loss: 0.0015652179718017578
backprop: 0.0017399787902832031
loss: 0.0015327930450439453
backprop: 0.0017604827880859375
loss: 0.00153350830078125
backprop: 0.00177001953125
loss: 0.0015811920166015625
backprop: 0.0017864704132080078
loss: 0.0015418529510498047
backprop: 0.0017881393432617188
loss: 0.0015835762023925781
backprop: 0.0017735958099365234
loss: 0.001608133316040039
backprop: 0.0017142295837402344
loss: 0.0015988349914550781
backprop: 0.001719713211059

...

loss: 0.0017209053039550781
backprop: 0.0015406608581542969
loss: 0.0018436908721923828
backprop: 0.002061128616333008
loss: 0.0017273426055908203
backprop: 0.0020301342010498047
loss: 0.001636505126953125
backprop: 0.0017099380493164062
loss: 0.0016069412231445312
backprop: 0.001916646957397461
loss: 0.0016362667083740234
backprop: 0.001688241958618164
loss: 0.0017833709716796875
backprop: 0.002516508102416992
loss: 0.0017294883728027344
backprop: 0.0016715526580810547
loss: 0.0018215179443359375
backprop: 0.0016486644744873047
loss: 0.0015938282012939453
backprop: 0.0023436546325683594
loss: 0.0017435550689697266
backprop: 0.0016727447509765625
loss: 0.001918792724609375
backprop: 0.001806020736694336
loss: 0.001608133316040039
backprop: 0.002430438995361328
loss: 0.0017518997192382812
backprop: 0.002099752426147461
loss: 0.0016548633575439453
backprop: 0.0016314983367919922
loss: 0.002239704132080078
backprop: 0.0018548965454101562
loss: 0.0015938282012939453
backprop: 0.00177717208

loss: 0.002889394760131836
backprop: 0.0020971298217773438
loss: 0.0016088485717773438
backprop: 0.0015828609466552734
loss: 0.001873016357421875
backprop: 0.0017979145050048828
loss: 0.001607656478881836
backprop: 0.0017061233520507812
loss: 0.0020668506622314453
backprop: 0.001588582992553711
loss: 0.0025815963745117188
backprop: 0.0015900135040283203
loss: 0.0016617774963378906
backprop: 0.0019648075103759766
loss: 0.0016050338745117188
backprop: 0.00205230712890625
loss: 0.001697540283203125
backprop: 0.0017561912536621094
loss: 0.0021495819091796875
backprop: 0.0016369819641113281
loss: 0.002031087875366211
backprop: 0.0015218257904052734
loss: 0.0016727447509765625
backprop: 0.0016875267028808594
loss: 0.0016012191772460938
backprop: 0.0017237663269042969
loss: 0.0017457008361816406
backprop: 0.0022499561309814453
loss: 0.0020248889923095703
backprop: 0.0019905567169189453
loss: 0.0018031597137451172
backprop: 0.0023381710052490234
loss: 0.0017266273498535156
backprop: 0.00230550

loss: 0.0017690658569335938
backprop: 0.0018317699432373047
loss: 0.0023984909057617188
backprop: 0.002842426300048828
loss: 0.001916646957397461
backprop: 0.002251863479614258
loss: 0.0016536712646484375
backprop: 0.002056121826171875
loss: 0.0020258426666259766
backprop: 0.0018856525421142578
loss: 0.0017066001892089844
backprop: 0.0018157958984375
loss: 0.0016026496887207031
backprop: 0.0017733573913574219
loss: 0.0016241073608398438
backprop: 0.0018644332885742188
loss: 0.001827239990234375
backprop: 0.0020427703857421875
loss: 0.002237081527709961
backprop: 0.0027017593383789062
loss: 0.002304553985595703
backprop: 0.002370119094848633
loss: 0.0023086071014404297
backprop: 0.0022258758544921875
loss: 0.0015654563903808594
backprop: 0.0017652511596679688
loss: 0.0017342567443847656
backprop: 0.00177001953125
loss: 0.0016040802001953125
backprop: 0.0016672611236572266
loss: 0.001603841781616211
backprop: 0.0016641616821289062
loss: 0.001650094985961914
backprop: 0.001759529113769531

loss: 0.0019071102142333984
backprop: 0.002021312713623047
loss: 0.001775979995727539
backprop: 0.002233266830444336
loss: 0.0016484260559082031
backprop: 0.0015826225280761719
loss: 0.0022814273834228516
backprop: 0.0020275115966796875


KeyboardInterrupt: 